# Homework 1 - Natural Language Processing

**Student Name:** Nidhin Ninan 

**Student UIN:** 700772413 

**Date:** Sept 15, 2025 

**Course:** CS 5720, Natural Language Processing

This notebook contains solutions to Homework 1 covering:
1. Regular Expressions
2. Tokenization 
3. Byte Pair Encoding (BPE)
4. Edit Distance

---


In [2]:
# Import necessary packages for the homework
import re
import string
from collections import Counter, defaultdict
from typing import List, Dict, Tuple, Set
import numpy as np
import pandas as pd

# For visualization (optional)
import matplotlib.pyplot as plt

# Set up display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("All packages imported successfully!")
print("Ready to solve Homework 1 problems.")


All packages imported successfully!
Ready to solve Homework 1 problems.


# Question 1: Regular Expressions

**Task:** Create regex patterns for various text matching requirements.

This section contains regex patterns for:
- U.S. ZIP codes (whole tokens only)
- Words that do NOT start with a capital letter
- Numbers with various formats (sign, thousands commas, decimal, scientific)
- Spelling variants of "email" (case insensitive)
- Interjections like "go", "goo", "gooo" with optional punctuation
- Lines ending with "?" followed by closing quotes/brackets/spaces


In [7]:
# Q1.1: U.S. ZIP codes (whole tokens only)
# Pattern matches 5 digits or 5 digits + hyphen/space + 4 digits
zip_pattern = r'\b\d{5}(?:[-\s]\d{4})?\b'
# zip_pattern = r'\b\d{5}([- ]\d{4})?\b'  #does work when finding the pattern but will not work when testing the test cases
                                        #using re.findall as it only return the last part of the pattern

print("Q1.1 - U.S. ZIP codes pattern:")
print(f"Pattern: {zip_pattern}")

# Test cases
test_zip_strings = [
    "12345",           # Valid: 5 digits
    "12345-6789",      # Valid: 5+4 format with hyphen
    "12345 6789",      # Valid: 5+4 format with space
]

print("\nTest results:")
for test_str in test_zip_strings:
    matches = re.findall(zip_pattern, test_str)
    print(f"'{test_str}' -> {matches}")


Q1.1 - U.S. ZIP codes pattern:
Pattern: \b\d{5}(?:[-\s]\d{4})?\b

Test results:
'12345' -> ['12345']
'12345-6789' -> ['12345-6789']
'12345 6789' -> ['12345 6789']


In [ ]:
# Q1.2: Words that do NOT start with a capital letter

# ASCII/simple English fallback
ascii_pattern = r'\b(?![A-Z])[a-z][a-z\'\-]*\b'

print("Q1.2 - Words NOT starting with capital letter:")
print(f"ASCII pattern: {ascii_pattern}")

# Test cases (using ASCII pattern for compatibility)
test_words = ["hello", "World", "don't", "state-of-the-art", "iPhone", "e-mail", "UNESCO"]
test_text = " ".join(test_words)

print(f"\nTest text: {test_text}")
matches = re.findall(ascii_pattern, test_text)
print(f"Words NOT starting with capital: {matches}")


Q1.2 - Words NOT starting with capital letter:
ASCII pattern: \b(?![A-Z])[a-z][a-z\'\-]*\b

Test text: hello World don't state-of-the-art iPhone e-mail UNESCO
Words NOT starting with capital: ['hello', "don't", 'state-of-the-art', 'e-mail']


In [44]:
# Q1.3: Numbers (sign, thousands commas, decimal, scientific notation)
number_pattern = r'(?<!\S)[+-]?(?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d+)?(?:[eE][+-]?\d+)?(?!\S)'

print("Q1.3 - Number pattern:")
print(f"Pattern: {number_pattern}")

# Test cases
test_numbers = [
    "42",
    "-12,345.67",
    "+3.2e-4",
    "1,000",
    "3.14159",
    "6.022e23",
    "-2.5E-10",
    "abc123",  # Should not match (embedded)
    "123abc"   # Should not match (embedded)
]

print("\nTest results:")
for test_num in test_numbers:
    matches = re.findall(number_pattern, test_num)
    print(f"'{test_num}' -> {matches}")

print("\nExplanation:")
print("- (?<!\\S): not preceded by non-whitespace (token boundary)")
print("- [+-]?: optional sign")
print("- (?:\\d{{1,3}}(?:,\\d{{3}})*|\\d+): thousands format OR regular digits")
print("\n\t- \\d{1,3}: This part matches the beginning of the number. It requires the number to start with one, two, or three digits before the first comma. This handles 1, 12, and 123.")
print("\t- (?:,\\d{3}): This is a non-capturing group that matches a \"chunk\" of a comma followed by exactly three digits (like ,000 or ,456).")
print("\t\t*This is the most important part for your question. It's a quantifier that applies to the entire group before it (?:,\\d{3}). It means match the preceding group zero or more times.")
print("\n- (?:\\.\\d+)?: optional decimal part")
print("- (?:[eE][+-]?\\d+)?: optional scientific notation")
print("- (?!\\S): not followed by non-whitespace (token boundary)")

Q1.3 - Number pattern:
Pattern: (?<!\S)[+-]?(?:\d{1,3}(?:,\d{3})*|\d+)(?:\.\d+)?(?:[eE][+-]?\d+)?(?!\S)

Test results:
'42' -> ['42']
'-12,345.67' -> ['-12,345.67']
'+3.2e-4' -> ['+3.2e-4']
'1,000' -> ['1,000']
'3.14159' -> ['3.14159']
'6.022e23' -> ['6.022e23']
'-2.5E-10' -> ['-2.5E-10']
'abc123' -> []
'123abc' -> []

Explanation:
- (?<!\S): not preceded by non-whitespace (token boundary)
- [+-]?: optional sign
- (?:\d{{1,3}}(?:,\d{{3}})*|\d+): thousands format OR regular digits

	- \d{1,3}: This part matches the beginning of the number. It requires the number to start with one, two, or three digits before the first comma. This handles 1, 12, and 123.
	- (?:,\d{3}): This is a non-capturing group that matches a "chunk" of a comma followed by exactly three digits (like ,000 or ,456).
		*This is the most important part for your question. It's a quantifier that applies to the entire group before it (?:,\d{3}). It means match the preceding group zero or more times.

- (?:\.\d+)?: optional 

In [47]:
# Q1.4: Spelling variants of "email" (case insensitive)
email_pattern = r'\b[eE](?:[-– ])?[mM][aA][iI][lL]\b'

print("Q1.4 - Email spelling variants pattern:")
print(f"Pattern: {email_pattern}")
print("- Can also use with re.IGNORECASE flag for case insensitivity")

# Test cases
test_email_variants = [
    "email",
    "e-mail", 
    "e mail",
    "e–mail",  # en-dash
    "Email",   # Capital E
    "E-MAIL",  # All caps
    "emails",  # Should not match (plural)
    "mail"   # Should not match
]

print("\nTest results:")
for variant in test_email_variants:
    matches = re.findall(email_pattern, variant) # ,Can also use re.IGNORECASE, (...,re.IGNORECASE) )
    print(f"'{variant}' -> {matches}")


Q1.4 - Email spelling variants pattern:
Pattern: \b[eE](?:[-– ])?[mM][aA][iI][lL]\b
- Can also usewith re.IGNORECASE flag for case insensitivity

Test results:
'email' -> ['email']
'e-mail' -> ['e-mail']
'e mail' -> ['e mail']
'e–mail' -> ['e–mail']
'Email' -> ['Email']
'E-MAIL' -> ['E-MAIL']
'emails' -> []
'mail' -> []


In [48]:
# Q1.5: Interjection "go", "goo", "gooo", etc. with optional punctuation
go_pattern = r'(?<!\S)go+[!.,?]?(?!\S)'

print("Q1.5 - 'Go' interjection pattern:")
print(f"Pattern: {go_pattern}")

# Test cases
test_go_variants = [
    "go",
    "goo!",
    "gooo?",
    "goooo.",
    "go,",
    "going",    # Should not match
    "ago",      # Should not match  
    "Let's go!",
    "Goooo!"
]

print("\nTest results:")
for variant in test_go_variants:
    matches = re.findall(go_pattern, variant, re.IGNORECASE)
    print(f"'{variant}' -> {matches}")


Q1.5 - 'Go' interjection pattern:
Pattern: (?<!\S)go+[!.,?]?(?!\S)

Test results:
'go' -> ['go']
'goo!' -> ['goo!']
'gooo?' -> ['gooo?']
'goooo.' -> ['goooo.']
'go,' -> ['go,']
'going' -> []
'ago' -> []
'Let's go!' -> ['go!']
'Goooo!' -> ['Goooo!']


In [49]:
# Q1.6: Lines ending with "?" followed by closing quotes/brackets and spaces
line_end_pattern = r'\?[ \t\"\')\]\}\"\'»]*$'

print("Q1.6 - Lines ending with '?' pattern:")
print(f"Pattern: {line_end_pattern}")
print("- Can use with re.MULTILINE flag")

# Test cases (multiline string)
test_lines = """Are you coming?
Are you coming?"
Are you coming?'
Are you coming?)
Are you coming?]
Are you coming? 
Are you coming?	
What about this? No.
How are you? Fine."""

print("\nTest results:")
print("Lines that match the pattern:")
for i, line in enumerate(test_lines.split('\n'), 1):
    if re.search(line_end_pattern, line):
        print(f"Line {i}: '{line}' ✓")
    else:
        print(f"Line {i}: '{line}' ✗")


Q1.6 - Lines ending with '?' pattern:
Pattern: \?[ \t\"\')\]\}\"\'»]*$
- Can use with re.MULTILINE flag

Test results:
Lines that match the pattern:
Line 1: 'Are you coming?' ✓
Line 2: 'Are you coming?"' ✓
Line 3: 'Are you coming?'' ✓
Line 4: 'Are you coming?)' ✓
Line 5: 'Are you coming?]' ✓
Line 6: 'Are you coming? ' ✓
Line 7: 'Are you coming?	' ✓
Line 8: 'What about this? No.' ✗
Line 9: 'How are you? Fine.' ✗


# Question 2: Tokenization

**Task:** Implement and compare different tokenization methods.

This section covers:
1. **Naïve tokenization** (space-based splitting)
2. **Manual tokenization** (hand-crafted rules)
3. **Tool-based tokenization** (regex-based tokenizer)
4. **Multiword Expression (MWE) identification**
5. **Comparison and reflection**

**Test paragraph:**
> "Yesterday I couldn't finish the report — it was a state-of-the-art model, but the dataset didn't cooperate. I emailed Dr. O'Neil at 3:00 p.m., and she'll review it tomorrow. The newer models outperform older ones; however, we're still testing and refining."


In [3]:
# Define the test paragraph for tokenization (Malayalam)
paragraph = """ഇന്നലെ എനിക്ക് റിപ്പോർട്ട് പൂർത്തിയാക്കാൻ കഴിഞ്ഞില്ല — അത് അത്യാധുനിക മോഡൽ ആയിരുന്നു, പക്ഷേ ഡാറ്റാസെറ്റ് സഹകരിച്ചില്ല. ഞാൻ ഡോ. കെ.എം. നായരിന് 3:00 പി.എം.-ന് ഇമെയിൽ അയച്ചു, അവർ നാളെ അത് പരിശോധിക്കും. പുതിയ മോഡലുകൾ പഴയവയേക്കാൾ മികച്ചതാണ്; എന്നിരുന്നാലും, ഞങ്ങൾ ഇപ്പോഴും പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്."""
English_Translation = """Yesterday I could not complete the report — it was a state-of-the-art model, but the dataset did not cooperate. I emailed Dr. K.M. Nair at 3:00 PM, they will check it tomorrow. The new models are better than the old ones; however, we are still testing."""

print("Test paragraph:")
print(f'"{paragraph}"')

print("English Translation")
print(f'"{English_Translation}"')
print(f"\nLength: {len(paragraph)} characters")


Test paragraph:
"ഇന്നലെ എനിക്ക് റിപ്പോർട്ട് പൂർത്തിയാക്കാൻ കഴിഞ്ഞില്ല — അത് അത്യാധുനിക മോഡൽ ആയിരുന്നു, പക്ഷേ ഡാറ്റാസെറ്റ് സഹകരിച്ചില്ല. ഞാൻ ഡോ. കെ.എം. നായരിന് 3:00 പി.എം.-ന് ഇമെയിൽ അയച്ചു, അവർ നാളെ അത് പരിശോധിക്കും. പുതിയ മോഡലുകൾ പഴയവയേക്കാൾ മികച്ചതാണ്; എന്നിരുന്നാലും, ഞങ്ങൾ ഇപ്പോഴും പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്."
English Translation
"Yesterday I could not complete the report — it was a state-of-the-art model, but the dataset did not cooperate. I emailed Dr. K.M. Nair at 3:00 PM, they will check it tomorrow. The new models are better than the old ones; however, we are still testing."

Length: 297 characters


In [4]:
# Q2.1: Naïve tokenization (space-based splitting)
def naive_tokenize(text: str) -> List[str]:
    """Simple space-based tokenization"""
    return text.split()

naive_tokens = naive_tokenize(paragraph)

print("Q2.1 - Naïve Tokenization (space-based):")
print(f"Method: text.split()")
print(f"Number of tokens: {len(naive_tokens)}")
print("First 10 tokens:", naive_tokens[:10])
print("Last 10 tokens:", naive_tokens[-10:])
print("\nCharacteristics:")
print("- Punctuation attached to neighboring words")
print("- Contractions kept intact")
print("- Simple and fast but crude")


Q2.1 - Naïve Tokenization (space-based):
Method: text.split()
Number of tokens: 33
First 10 tokens: ['ഇന്നലെ', 'എനിക്ക്', 'റിപ്പോർട്ട്', 'പൂർത്തിയാക്കാൻ', 'കഴിഞ്ഞില്ല', '—', 'അത്', 'അത്യാധുനിക', 'മോഡൽ', 'ആയിരുന്നു,']
Last 10 tokens: ['അത്', 'പരിശോധിക്കും.', 'പുതിയ', 'മോഡലുകൾ', 'പഴയവയേക്കാൾ', 'മികച്ചതാണ്;', 'എന്നിരുന്നാലും,', 'ഞങ്ങൾ', 'ഇപ്പോഴും', 'പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്.']

Characteristics:
- Punctuation attached to neighboring words
- Contractions kept intact
- Simple and fast but crude


In [18]:
# Q2.2: Manual tokenization (hand-crafted rules)
def manual_tokenize(text: str) -> List[str]:
    """
    Manual tokenization with specific rules:
    1. Separate general punctuation into own tokens
    2. Keep internal apostrophes/hyphens in words  
    3. Split clitics (could + n't, she + 'll, we + 're)
    NOTE: these rules are implemented as the HW asked use to implement the rules BUT malayalam language doesn't use these rules
    """
    
    # Start with naive split
    tokens = text.split()
    result = []
    
    for token in tokens:
        # Handle punctuation at the end
        while token and token[-1] in '.,;:!?"—':
            if len(token) == 1:
                result.append(token)
                token = ""
                break
            else:
                # Remove punctuation and add it separately
                result.append(token[:-1])
                result.append(token[-1])
                token = ""  # Set to empty so we don't process it again
                break  # Exit the while loop
        
        if not token:  # If token is empty, skip the rest
            continue
            
        # Handle specific contractions/clitics (only if token still has content)
        if token.endswith("n't"):
            result.extend([token[:-3], "n't"])
        elif token.endswith("'ll"):
            result.extend([token[:-3], "'ll"])
        elif token.endswith("'re"):
            result.extend([token[:-3], "'re"])
        elif token.endswith("'ve"):
            result.extend([token[:-3], "'ve"])
        elif token.endswith("'d"):
            result.extend([token[:-2], "'d"])
        else:
            result.append(token)
    
    return result

manual_tokens = manual_tokenize(paragraph)

print("Q2.2 - Manual Tokenization:")
print(f"Number of tokens: {len(manual_tokens)}")
print("First 15 tokens:", manual_tokens[:15])
print("Last 15 tokens:", manual_tokens[-15:])
print("\nKey features:")
print(" NOTE: these rules are implemented as the HW asked use to implement the rules BUT malayalam language doesn't use ALL these rule")
print("\n- Separates punctuation into own tokens")
print("- Splits clitics (couldn't -> could + n't)")
print("- Keeps internal apostrophes/hyphens (state-of-the-art)")

Q2.2 - Manual Tokenization:
Number of tokens: 42
First 15 tokens: ['ഇന്നലെ', 'എനിക്ക്', 'റിപ്പോർട്ട്', 'പൂർത്തിയാക്കാൻ', 'കഴിഞ്ഞില്ല', '—', 'അത്', 'അത്യാധുനിക', 'മോഡൽ', 'ആയിരുന്നു', ',', 'പക്ഷേ', 'ഡാറ്റാസെറ്റ്', 'സഹകരിച്ചില്ല', '.']
Last 15 tokens: ['നാളെ', 'അത്', 'പരിശോധിക്കും', '.', 'പുതിയ', 'മോഡലുകൾ', 'പഴയവയേക്കാൾ', 'മികച്ചതാണ്', ';', 'എന്നിരുന്നാലും', ',', 'ഞങ്ങൾ', 'ഇപ്പോഴും', 'പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്', '.']

Key features:
 NOTE: these rules are implemented as the HW asked use to implement the rules BUT malayalam language doesn't use ALL these rule

- Separates punctuation into own tokens
- Splits clitics (couldn't -> could + n't)
- Keeps internal apostrophes/hyphens (state-of-the-art)


## NOTE: 

**The following few cells are included in the notebook as they were required to experiment and install the necessary packages and dependencies for doing tokenisation in the jlab_server environment (Python 3.12.5) that this Jupyter notebook was developed in. RUN THEM ONLY IF NEEDED**

In [ ]:
# # Setup for iNLTK Library (run this cell first)
# import sys
# import os
# import subprocess

# # Install iNLTK if not already installed
# try:
#     import inltk
#     print("iNLTK is already installed")
# except ImportError:
#     print("Installing iNLTK...")
#     subprocess.check_call([
#         sys.executable, "-m", "pip", "install", "inltk",
#         "--timeout", "1800",  # 30 minutes timeout
#         "--retries", "3",     # Retry 3 times if failed
#         "--no-cache-dir"      # Don't cache to save space
#     ])
#     print("iNLTK installation completed!")

# # Download Malayalam language model for iNLTK
# try:
#     from inltk.inltk import setup
#     print("Downloading Malayalam language model...")
#     setup('ml')  # 'ml' is the language code for Malayalam
#     print("Malayalam model downloaded successfully!")
# except Exception as e:
#     print(f"Error downloading Malayalam model: {e}")
#     print("You may need to run this cell again or check your internet connection.")

# print("iNLTK setup complete!")


iNLTK is already installed
Error downloading Malayalam model: cannot import name 'Iterable' from 'collections' (/home/nidhinninan/.config/jupyterlab-desktop/jlab_server/lib/python3.12/collections/__init__.py)
You may need to run this cell again or check your internet connection.
iNLTK setup complete!


In [36]:
# import subprocess
# import sys

# # Install transformers if not available
# try:
#     from transformers import AutoTokenizer
#     print("✓ Transformers already available")
# except ImportError:
#     print("Installing transformers...")
#     subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"], timeout=1800)
#     from transformers import AutoTokenizer
#     print("✓ Transformers installed successfully")

✓ Transformers already available


In [12]:
# # Modern Malayalam Tokenizers - Python 3.12 Compatible

# import sys
# import subprocess

# def setup_huggingface_tokenizer():
#     """Setup HuggingFace tokenizer - BEST option"""
#     try:
#         from transformers import AutoTokenizer
#         print("✓ Transformers already available")
#     except ImportError:
#         print("Installing transformers...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"])
#         from transformers import AutoTokenizer
    
#     # Try different multilingual models that support Malayalam
#     models_to_try = [
#         "google/muril-base-cased",  # Multilingual model with Malayalam support
#         "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",  # Multilingual
#         "xlm-roberta-base",  # Covers many languages including Malayalam
#     ]
    
#     for model_name in models_to_try:
#         try:
#             print(f"Trying {model_name}...")
#             tokenizer = AutoTokenizer.from_pretrained(model_name)
            
#             # Test with Malayalam
#             malayalam_text = "ഇന്നലെ എനിക്ക് റിപ്പോർട്ട് പൂർത്തിയാക്കാൻ കഴിഞ്ഞില്ല"
#             tokens = tokenizer.tokenize(malayalam_text)
#             print(f"✓ {model_name} works! Sample tokens: {tokens[:5]}...")
            
#             def tokenize_malayalam(text):
#                 return tokenizer.tokenize(text)
            
#             return tokenize_malayalam, f"HuggingFace {model_name}"
            
#         except Exception as e:
#             print(f"✗ {model_name} failed: {str(e)[:50]}...")
#             continue
    
#     return None, None

# def setup_sentencepiece_tokenizer():
#     """Setup SentencePiece tokenizer - you already have this installed!"""
#     try:
#         import sentencepiece as spm
#         print("✓ SentencePiece already available")
        
#         # Create a simple BPE tokenizer for Malayalam
#         # Note: This is a basic setup - for production, you'd train on Malayalam corpus
        
#         def basic_sentencepiece_tokenize(text):
#             # Simple character-level tokenization for Malayalam
#             # In production, you'd have a trained Malayalam model
#             chars = list(text)
#             # Group Malayalam characters, split others
#             tokens = []
#             current_token = ""
            
#             for char in chars:
#                 if '\u0D00' <= char <= '\u0D7F':  # Malayalam range
#                     current_token += char
#                 else:
#                     if current_token:
#                         tokens.append(current_token)
#                         current_token = ""
#                     if not char.isspace():
#                         tokens.append(char)
            
#             if current_token:
#                 tokens.append(current_token)
            
#             return tokens
        
#         # Test it
#         malayalam_text = "ഇന്നലെ എനിക്ക് റിപ്പോർട്ട്"
#         tokens = basic_sentencepiece_tokenize(malayalam_text)
#         print(f"✓ Basic SentencePiece working: {tokens}")
        
#         return basic_sentencepiece_tokenize, "SentencePiece (Basic)"
        
#     except Exception as e:
#         print(f"✗ SentencePiece failed: {e}")
#         return None, None

# def setup_nltk_tokenizer():
#     """Setup NLTK with Indic support"""
#     try:
#         import nltk
#         print("✓ NLTK available")
#     except ImportError:
#         print("Installing NLTK...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", "nltk"])
#         import nltk
    
#     try:
#         # Download required NLTK data
#         nltk.download('punkt', quiet=True)
        
#         from nltk.tokenize import word_tokenize
#         import re
        
#         def nltk_malayalam_tokenize(text):
#             # Use NLTK's word_tokenize and then clean up for Malayalam
#             tokens = word_tokenize(text)
            
#             # Post-process to handle Malayalam better
#             clean_tokens = []
#             for token in tokens:
#                 # If token contains Malayalam, keep as is
#                 if re.search(r'[\u0D00-\u0D7F]', token):
#                     clean_tokens.append(token)
#                 # Otherwise, use NLTK's tokenization
#                 else:
#                     clean_tokens.append(token)
            
#             return clean_tokens
        
#         # Test it
#         malayalam_text = "ഇന്നലെ എനിക്ക് റിപ്പോർട്ട് പൂർത്തിയാക്കാൻ കഴിഞ്ഞില്ല"
#         tokens = nltk_malayalam_tokenize(malayalam_text)
#         print(f"✓ NLTK Malayalam working: {tokens[:5]}...")
        
#         return nltk_malayalam_tokenize, "NLTK + Malayalam"
        
#     except Exception as e:
#         print(f"✗ NLTK setup failed: {e}")
#         return None, None

# def setup_best_malayalam_tokenizer():
#     """Try tokenizers in order of preference"""
    
#     print("=== Setting up Malayalam Tokenization ===")
    
#     # Order of preference
#     tokenizer_setups = [
#         ("HuggingFace Transformers", setup_huggingface_tokenizer),
#         ("NLTK", setup_nltk_tokenizer),
#         ("SentencePiece", setup_sentencepiece_tokenizer),
#     ]
    
#     for name, setup_func in tokenizer_setups:
#         print(f"\n--- Trying {name} ---")
#         tokenizer, method = setup_func()
        
#         if tokenizer:
#             print(f"✓ Success with {method}")
#             return tokenizer, method
    
#     print("\n✗ All modern tokenizers failed")
#     return None, None

# # Main execution
# print("Setting up modern Malayalam tokenization...")
# tokenize_malayalam, method = setup_best_malayalam_tokenizer()

# if tokenize_malayalam:
#     # Test with various texts
#     test_texts = [
#         "ഇന്നലെ എനിക്ക് റിപ്പോർട്ട് പൂർത്തിയാക്കാൻ കഴിഞ്ഞില്ല",
#         "നമസ്കാരം! എങ്ങനെയുണ്ട്?",
#         "കേരളം എന്നറിയപ്പെടുന്നത് God's Own Country എന്നാണ്."
#     ]
    
#     print(f"\n=== Testing {method} ===")
#     for i, text in enumerate(test_texts, 1):
#         tokens = tokenize_malayalam(text)
#         print(f"\nTest {i}:")
#         print(f"Text: {text}")
#         print(f"Tokens: {tokens}")
#         print(f"Count: {len(tokens)}")
    
#     print(f"\n✅ Malayalam tokenization ready using {method}!")
#     print("Use tokenize_malayalam(text) function for your tokenization needs.")
    
# else:
#     print("\n❌ No modern tokenizers worked")
#     print("Consider using the simple regex-based approach from earlier")

Setting up modern Malayalam tokenization...
=== Setting up Malayalam Tokenization ===

--- Trying HuggingFace Transformers ---
✓ Transformers already available
Trying google/muril-base-cased...
✓ google/muril-base-cased works! Sample tokens: ['ഇന്നലെ', 'എനിക്ക്', 'റിപ്പോർട്ട്', 'പൂർത്തിയാക്കാൻ', 'കഴിഞ്ഞില്ല']...
✓ Success with HuggingFace google/muril-base-cased

=== Testing HuggingFace google/muril-base-cased ===

Test 1:
Text: ഇന്നലെ എനിക്ക് റിപ്പോർട്ട് പൂർത്തിയാക്കാൻ കഴിഞ്ഞില്ല
Tokens: ['ഇന്നലെ', 'എനിക്ക്', 'റിപ്പോർട്ട്', 'പൂർത്തിയാക്കാൻ', 'കഴിഞ്ഞില്ല']
Count: 5

Test 2:
Text: നമസ്കാരം! എങ്ങനെയുണ്ട്?
Tokens: ['ന', '##മസ്കാരം', '!', 'എങ്ങനെ', '##യുണ്ട്', '?']
Count: 6

Test 3:
Text: കേരളം എന്നറിയപ്പെടുന്നത് God's Own Country എന്നാണ്.
Tokens: ['കേരളം', 'എന്നറിയപ്പെടുന്ന', '##ത്', 'God', "'", 's', 'Own', 'Country', 'എന്നാണ്', '.']
Count: 10

✅ Malayalam tokenization ready using HuggingFace google/muril-base-cased!
Use tokenize_malayalam(text) function for your tokenization needs.


In [42]:
# # Simple fix for iNLTK import issue
# import sys
# import subprocess

# def setup_inltk():
#     try:
#         # Try importing first
#         from inltk.inltk import setup, tokenize
#         setup('ml')
#         return True
#     except ImportError:
#         print("Installing iNLTK...")
#         subprocess.check_call([sys.executable, "-m", "pip", "install", "inltk"], timeout=1800)
#         try:
#             from inltk.inltk import setup, tokenize
#             setup('ml')
#             return True
#         except:
#             return False
#     except Exception as e:
#         print(f"iNLTK setup failed: {e}")
#         return False

# # Try setup
# inltk_ready = setup_inltk()
# if inltk_ready:
#     print("✓ iNLTK ready for Malayalam")
# else:
#     print("⚠️ Will use fallback tokenization")

In [ ]:
# # Fix for iNLTK sklearn dependency issue
# import sys
# import subprocess

# def install_sklearn_simple():
#     """Try to install sklearn with a simple approach"""
#     try:
#         import sklearn
#         print("✓ sklearn already available")
#         return True
#     except ImportError:
#         try:
#             print("Installing sklearn (this may take a moment)...")
#             # Try a simpler, lighter installation
#             subprocess.check_call([
#                 sys.executable, "-m", "pip", "install", 
#                 "scikit-learn==1.0.2", "--no-deps"
#             ], timeout=1800)
#             print("✓ sklearn installed successfully")
#             return True
#         except:
#             print("❌ Could not install sklearn automatically")
#             return False

# # Try to fix the sklearn issue
# sklearn_available = install_sklearn_simple()

# # Now try iNLTK Malayalam setup again
# if sklearn_available:
#     try:
#         from inltk.inltk import setup, tokenize
#         print("Attempting Malayalam model setup...")
#         setup('ml')
#         print("✓ Malayalam model setup successful!")
    
#         # Test it
#         test_tokens = tokenize("ഇന്നലെ എനിക്ക് കഴിഞ്ഞില്ല", 'ml')
#         print(f"✓ Test successful: {test_tokens}")
        
#     except Exception as e:
#         print(f"⚠️ iNLTK setup issue: {str(e)}")
#         print("Will use fallback tokenization")
# else:
#     print("⚠️ Using fallback approach due to dependency issues")

In [ ]:
# print('Installing missing dependencies for iNLTK...')

# # Install scikit-learn (sklearn)
# try:
#     subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn'], timeout=1800)
#     print('✓ scikit-learn installed')
# except Exception as e:
#     print(f'Error installing scikit-learn: {e}')

# # Also install other common dependencies that might be missing
# dependencies = ['torch', 'numpy', 'pandas']
# for dep in dependencies:
#     try:
#         __import__(dep)
#         print(f'✓ {dep} already available')
#     except ImportError:
#         try:
#             print(f'Installing {dep}...')
#             subprocess.check_call([sys.executable, '-m', 'pip', 'install', dep], timeout=120)
#             print(f'✓ {dep} installed')
#         except Exception as e:
#             print(f'Warning: Could not install {dep}: {e}')

# print('Dependencies installation complete!')

**ALL NECESSARY PACKAGES INSTALLED ABOVE**

In [14]:
# Q2.3: Tool-based tokenization using google/muril-base-cased
from transformers import AutoTokenizer

# Initialize the google/muril-base-cased tokenizer
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

def malayalam_tokenize_text(text: str) -> List[str]:
    """
    Tool-based Malayalam tokenizer using google/muril-base-cased:
    - Uses pre-trained multilingual BERT model with Malayalam support
    - Handles subword tokenization with WordPiece
    - Works with mixed Malayalam-English text
    - Produces linguistically meaningful subword units
    """
    
    # Use the muril tokenizer to tokenize the text
    tokens = muril_tokenizer.tokenize(text)
    
    return tokens

tool_tokens = malayalam_tokenize_text(paragraph)

print("Q2.3 - Tool-based Tokenization using google/muril-base-cased:")
print(f"Using tokenizer: google/muril-base-cased (Multilingual BERT)")
print(f"Number of tokens: {len(tool_tokens)}")
print("First 15 tokens:", tool_tokens[:15])
print("Last 15 tokens:", tool_tokens[-15:])
print("\nKey features:")
print("- Uses google/muril-base-cased pre-trained model")
print("- WordPiece subword tokenization")
print("- Handles Malayalam Unicode and complex scripts")
print("- Produces linguistically meaningful subword units")
print("- Supports mixed Malayalam-English text")
print("- Uses ## prefix for subword continuation")

Q2.3 - Tool-based Tokenization using google/muril-base-cased:
Using tokenizer: google/muril-base-cased (Multilingual BERT)
Number of tokens: 70
First 15 tokens: ['ഇന്നലെ', 'എനിക്ക്', 'റിപ്പോർട്ട്', 'പൂർത്തിയാക്കാൻ', 'കഴിഞ്ഞില്ല', '—', 'അത്', 'അത്', '##യാ', '##ധുനിക', 'മോഡൽ', 'ആയിരുന്നു', ',', 'പക്ഷേ', 'ഡാറ്റാ']
Last 15 tokens: ['##യേക്കാൾ', 'മികച്ച', '##താണ്', ';', 'എന്നിരുന്നാലും', ',', 'ഞങ്ങൾ', 'ഇപ്പോഴും', 'പരീക്ഷ', '##ിച്ചു', '##ക', '##ൊണ്ട', '##ിരി', '##ക്കുകയാണ്', '.']

Key features:
- Uses google/muril-base-cased pre-trained model
- WordPiece subword tokenization
- Handles Malayalam Unicode and complex scripts
- Produces linguistically meaningful subword units
- Supports mixed Malayalam-English text
- Uses ## prefix for subword continuation


In [20]:
# Q2.4: Comparison of tokenization methods
def compare_tokenizations():
    """Compare the three tokenization methods"""
    
    print("=== TOKENIZATION COMPARISON ===\n")
    
    methods = [
        ("Naïve (space)", naive_tokens),
        ("Manual (rules)", manual_tokens), 
        ("Tool (google/muril)", tool_tokens)
    ]
    
    for name, tokens in methods:
        print(f"{name}: {len(tokens)} tokens")
    
    print("\n=== KEY DIFFERENCES ===")
    
    # Find differences between manual and tool tokenization
    manual_set = set(manual_tokens)
    tool_set = set(tool_tokens)
    
    print(f"\nTokens in manual but not in tool: {len(manual_set - tool_set)} unique tokens")
    print(f"Tokens in tool but not in manual: {len(tool_set - manual_set)} unique tokens")
    
    # Show sample differences (first 10 of each)
    manual_only = list(manual_set - tool_set)[:10]
    tool_only = list(tool_set - manual_set)[:10]
    
    print(f"\nSample tokens only in manual: {manual_only}")
    print(f"Sample tokens only in tool: {tool_only}")
    
    # Analyze specific Malayalam words and their tokenization
    malayalam_examples = ["ഇന്നലെ", "പൂർത്തിയാക്കാൻ", "അത്യാധുനിക", "സഹകരിച്ചില്ല", "പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്"]
    
    # Analyze specific Malayalam words using each tokenization method directly
    malayalam_examples = ["പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്."]

    print("\n=== MALAYALAM WORD TOKENIZATION EXAMPLES ===")
    for example in malayalam_examples:
        print(f"\nSample word: '{example}'")
        
        # Apply each tokenization method directly to the sample word
        naive_result = naive_tokenize(example)
        manual_result = manual_tokenize(example) 
        tool_result = malayalam_tokenize_text(example)
        
        print(f"  Naïve tokenization:   {naive_result}")
        print(f"  Manual tokenization:  {manual_result}")
        print(f"  Tool tokenization:    {tool_result}")
        
        # Show token counts
        print(f"  Token counts: Naïve={len(naive_result)}, Manual={len(manual_result)}, Tool={len(tool_result)}")

compare_tokenizations()

=== TOKENIZATION COMPARISON ===

Naïve (space): 33 tokens
Manual (rules): 42 tokens
Tool (google/muril): 70 tokens

=== KEY DIFFERENCES ===

Tokens in manual but not in tool: 11 unique tokens
Tokens in tool but not in manual: 34 unique tokens

Sample tokens only in manual: ['അത്യാധുനിക', 'മികച്ചതാണ്', 'ഡാറ്റാസെറ്റ്', 'കെ.എം', 'പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്', 'പി.എം.-ന്', 'പഴയവയേക്കാൾ', 'സഹകരിച്ചില്ല', '3:00', 'പരിശോധിക്കും']
Sample tokens only in tool: ['മികച്ച', '##ധുനിക', '##ധി', 'പഴയ', '##താണ്', '##ൊണ്ട', '##ിച്ചു', '##ക്കുകയാണ്', '##യേക്കാൾ', '##ക']

=== MALAYALAM WORD TOKENIZATION EXAMPLES ===

Sample word: 'പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്.'
  Naïve tokenization:   ['പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്.']
  Manual tokenization:  ['പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്', '.']
  Tool tokenization:    ['പരീക്ഷ', '##ിച്ചു', '##ക', '##ൊണ്ട', '##ിരി', '##ക്കുകയാണ്', '.']
  Token counts: Naïve=1, Manual=2, Tool=7


In [23]:
# Q2.5: Multiword Expressions (MWEs) identification
def identify_mwes(text: str) -> List[Tuple[str, str]]:
    """
    Identify multiword expressions in the Malayalam text and explain why they should be 
    treated as single tokens.
    """
    
    mwes = [
        ("അത്യാധുനിക മോഡൽ", "Fixed adjectival compound; semantic unity (state-of-the-art model)"),
        ("ഡോ. കെ.എം. നായരിന്", "Person's title + name; named entity (Dr. K.M. Nair)"),
        ("3:00 പി.എം.", "Time expression; numeric + temporal unit (3:00 PM)"),
        ("പുതിയ മോഡലുകൾ", "Adjective-noun compound; semantic unity (new models)"),
        ("പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്", "Complex verbal form; single semantic unit (are testing)"),
    ]
    
    return mwes

mwes = identify_mwes(paragraph)

print("Q2.5 - Multiword Expressions (MWEs) Identified v2:")
print(f"Found {len(mwes)} MWEs in the Malayalam paragraph:\n")

for i, (mwe, reason) in enumerate(mwes, 1):
    print(f"{i}. '{mwe}'")
    print(f"   Reason: {reason}")
    # Check presence more carefully for Malayalam text
    mwe_clean = mwe.replace('.', '').replace(' ', '')
    text_clean = paragraph.replace(' ', '')
    present = mwe_clean in text_clean or any(word in paragraph for word in mwe.split())
    print(f"   Present in text: {'✓' if present else '✗'}")
    print()

print("WHY TREAT AS SINGLE TOKENS:")
print("- MWEs are semantically or syntactically atomic units")
print("- Preserves meaning and reduces fragmentation in analysis") 
print("- Helps downstream NLP tasks (NER, translation, sentiment analysis)")
print("- Avoids losing fixed phrasal meanings and cultural concepts")
print("- Particularly important for Malayalam compound verbs and complex expressions")

# Show how google/muril-base-cased handles these MWEs
print(f"\n=== HOW GOOGLE/MURIL HANDLES MWES ===")
for mwe, reason in mwes[:3]:  # Show first 3 examples
    tokens = muril_tokenizer.tokenize(mwe)
    print(f"'{mwe}' → {tokens}")
print("Note: Subword tokenization may split MWEs, but preserves morphological information")

Q2.5 - Multiword Expressions (MWEs) Identified v2:
Found 5 MWEs in the Malayalam paragraph:

1. 'അത്യാധുനിക മോഡൽ'
   Reason: Fixed adjectival compound; semantic unity (state-of-the-art model)
   Present in text: ✓

2. 'ഡോ. കെ.എം. നായരിന്'
   Reason: Person's title + name; named entity (Dr. K.M. Nair)
   Present in text: ✓

3. '3:00 പി.എം.'
   Reason: Time expression; numeric + temporal unit (3:00 PM)
   Present in text: ✓

4. 'പുതിയ മോഡലുകൾ'
   Reason: Adjective-noun compound; semantic unity (new models)
   Present in text: ✓

5. 'പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്'
   Reason: Complex verbal form; single semantic unit (are testing)
   Present in text: ✓

WHY TREAT AS SINGLE TOKENS:
- MWEs are semantically or syntactically atomic units
- Preserves meaning and reduces fragmentation in analysis
- Helps downstream NLP tasks (NER, translation, sentiment analysis)
- Avoids losing fixed phrasal meanings and cultural concepts
- Particularly important for Malayalam compound verbs and complex expressi

## Q2.6: Reflection on Tokenization

**Observation based on the above tokenisation of sentences in Malayalam:**

1. **WordPiece vs. Rule-based Tokenization:** The google/muril-base-cased model using WordPiece tokenization handles the Malayalam paragraph much more effectively than naive or manual approaches, breaking complex Malayalam words into meaningful subword units marked with ## continuation symbols that preserve morphological information.

2. **Malayalam Script Complexity:** Malayalam's agglutinative nature and complex script with conjuncts, vowel signs, and compound words pose significant challenges for simple tokenization methods, but WordPiece tokenization trained on multilingual data can capture these linguistic patterns more naturally.

3. **Subword Granularity Benefits:** The ## prefix system in WordPiece allows the model to represent long Malayalam words (like "പരീക്ഷിച്ചുകൊണ്ടിരിക്കുകയാണ്") as sequences of meaningful subword pieces, enabling better handling of unseen words and morphological variations compared to whole-word approaches.

4. **Cross-script Robustness:** When processing mixed Malayalam-English text in our paragraph, the pre-trained google/muril model seamlessly handles both scripts without requiring separate tokenization rules, demonstrating the advantage of multilingual models over language-specific manual rules.

5. **Trade-offs in Semantic Preservation:** While WordPiece tokenization excels at morphological decomposition and OOV handling, it may fragment semantically coherent units like proper names ("ഡോ. കെ.എം. നായരിന്") or compound expressions, requiring careful consideration for downstream tasks that depend on preserving semantic boundaries.

**Reflection on the Tokenisation based of the solution**:

The most challenging aspect of tokenizing Malayalam is its agglutinative morphology, where multiple suffixes and particles attach to a root word, creating long, complex forms that are difficult to segment correctly. Compared to English, which has more isolated word structures and clearer space delimiters, Malayalam tokenization must handle intricate word boundaries that aren't always separated by spaces. Punctuation, complex morphology, and multiword expressions (MWEs) significantly increase this difficulty, as they require a deeper linguistic understanding beyond simple rules to preserve the correct meaning of tokens.


# Question 3: Byte Pair Encoding (BPE)

**Task:** Implement BPE manually and programmatically, then analyze results.

This section covers:
1. **Manual BPE** on toy corpus (first 3 merges step-by-step)
2. **Programmatic BPE learner** on toy corpus (10 merges)
3. **BPE training** on the paragraph from Q2 (30 merges)
4. **Analysis and reflection** on subword tokenization

**Toy corpus:** 
`low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new`


In [24]:
# Define the toy corpus for BPE
toy_corpus = "low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new"

# Convert to word list and add end-of-word markers
words = toy_corpus.split()
word_counts = Counter(words)

print("Q3 - Toy Corpus Setup:")
print(f"Original corpus: {toy_corpus}")
print(f"Word counts: {dict(word_counts)}")

# Convert to character-level with end-of-word markers
def prepare_bpe_corpus(word_counts):
    """Prepare corpus for BPE by splitting into characters and adding end-of-word markers"""
    bpe_corpus = {}
    for word, count in word_counts.items():
        # Split into characters and add end-of-word marker
        char_word = ' '.join(list(word)) + ' _'
        bpe_corpus[char_word] = count
    return bpe_corpus

bpe_corpus = prepare_bpe_corpus(word_counts)

print(f"\nInitial BPE corpus (character-level with end-of-word markers):")
for word, count in bpe_corpus.items():
    print(f"'{word}': {count}")


Q3 - Toy Corpus Setup:
Original corpus: low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new
Word counts: {'low': 5, 'lowest': 2, 'newer': 6, 'wider': 3, 'new': 2}

Initial BPE corpus (character-level with end-of-word markers):
'l o w _': 5
'l o w e s t _': 2
'n e w e r _': 6
'w i d e r _': 3
'n e w _': 2


In [25]:
# Q3.1: Manual BPE - First 3 merges step by step
def manual_bpe_step_by_step():
    """Perform manual BPE step by step for the first 3 merges"""
    
    # Initial state
    current_corpus = bpe_corpus.copy()
    merge_history = []
    
    print("=== MANUAL BPE - STEP BY STEP ===\n")
    
    for step in range(1, 4):  # First 3 steps
        print(f"STEP {step}:")
        
        # Count all adjacent pairs
        pair_counts = defaultdict(int)
        for word, count in current_corpus.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pair = (symbols[i], symbols[i + 1])
                pair_counts[pair] += count
        
        if not pair_counts:
            break
            
        # Find most frequent pair
        most_frequent_pair = max(pair_counts.items(), key=lambda x: x[1])
        pair, freq = most_frequent_pair
        
        print(f"Most frequent pair: ({pair[0]}, {pair[1]}) with count {freq}")
        
        # Merge the pair
        new_symbol = pair[0] + pair[1]
        merge_history.append((pair[0], pair[1], new_symbol))
        
        # Update corpus
        new_corpus = {}
        for word, count in current_corpus.items():
            # Replace the pair in this word
            symbols = word.split()
            new_symbols = []
            i = 0
            while i < len(symbols):
                if i < len(symbols) - 1 and symbols[i] == pair[0] and symbols[i + 1] == pair[1]:
                    new_symbols.append(new_symbol)
                    i += 2
                else:
                    new_symbols.append(symbols[i])
                    i += 1
            new_word = ' '.join(new_symbols)
            new_corpus[new_word] = count
        
        current_corpus = new_corpus
        
        print(f"Merge: {pair[0]} + {pair[1]} → {new_symbol}")
        print("Updated corpus:")
        for word, count in current_corpus.items():
            print(f"  '{word}': {count}")
        print()
    
    return merge_history, current_corpus

manual_merges, manual_final_corpus = manual_bpe_step_by_step()


=== MANUAL BPE - STEP BY STEP ===

STEP 1:
Most frequent pair: (e, r) with count 9
Merge: e + r → er
Updated corpus:
  'l o w _': 5
  'l o w e s t _': 2
  'n e w er _': 6
  'w i d er _': 3
  'n e w _': 2

STEP 2:
Most frequent pair: (er, _) with count 9
Merge: er + _ → er_
Updated corpus:
  'l o w _': 5
  'l o w e s t _': 2
  'n e w er_': 6
  'w i d er_': 3
  'n e w _': 2

STEP 3:
Most frequent pair: (n, e) with count 8
Merge: n + e → ne
Updated corpus:
  'l o w _': 5
  'l o w e s t _': 2
  'ne w er_': 6
  'w i d er_': 3
  'ne w _': 2



In [26]:
# Q3.2: Programmatic BPE learner on toy corpus (10 merges)
class BPELearner:
    """Simple BPE learner implementation"""
    
    def __init__(self):
        self.merges = []
        self.vocab = set()
    
    def get_pairs(self, word_freqs):
        """Get all adjacent symbol pairs and their frequencies"""
        pairs = defaultdict(int)
        for word, freq in word_freqs.items():
            symbols = word.split()
            for i in range(len(symbols) - 1):
                pairs[(symbols[i], symbols[i + 1])] += freq
        return pairs
    
    def merge_vocab(self, pair, word_freqs):
        """Merge the most frequent pair in vocabulary"""
        new_word_freqs = {}
        bigram = ' '.join(pair)
        replacement = ''.join(pair)
        
        for word in word_freqs:
            new_word = word.replace(bigram, replacement)
            new_word_freqs[new_word] = word_freqs[word]
        
        return new_word_freqs
    
    def learn_bpe(self, word_freqs, num_merges):
        """Learn BPE merges from word frequencies"""
        self.merges = []
        current_word_freqs = word_freqs.copy()
        
        print(f"=== BPE LEARNER - {num_merges} MERGES ===\n")
        
        for i in range(num_merges):
            pairs = self.get_pairs(current_word_freqs)
            if not pairs:
                break
            
            # Get most frequent pair
            best_pair = max(pairs, key=pairs.get)
            freq = pairs[best_pair]
            
            print(f"Merge {i+1}: {best_pair[0]} + {best_pair[1]} -> {''.join(best_pair)} (freq: {freq})")
            
            # Apply merge
            current_word_freqs = self.merge_vocab(best_pair, current_word_freqs)
            self.merges.append(best_pair)
        
        print(f"\nFinal vocabulary after {len(self.merges)} merges:")
        for word, freq in sorted(current_word_freqs.items()):
            print(f"  '{word}': {freq}")
        
        return current_word_freqs

# Run BPE learner on toy corpus
bpe_learner = BPELearner()
final_toy_corpus = bpe_learner.learn_bpe(bpe_corpus, 10)


=== BPE LEARNER - 10 MERGES ===

Merge 1: e + r -> er (freq: 9)
Merge 2: er + _ -> er_ (freq: 9)
Merge 3: n + e -> ne (freq: 8)
Merge 4: ne + w -> new (freq: 8)
Merge 5: l + o -> lo (freq: 7)
Merge 6: lo + w -> low (freq: 7)
Merge 7: new + er_ -> newer_ (freq: 6)
Merge 8: low + _ -> low_ (freq: 5)
Merge 9: w + i -> wi (freq: 3)
Merge 10: wi + d -> wid (freq: 3)

Final vocabulary after 10 merges:
  'low e s t _': 2
  'low_': 5
  'new _': 2
  'newer_': 6
  'wid er_': 3


In [27]:
# Q3.2: Demonstrate BPE segmentation on examples
def apply_bpe_segmentation(word, merges):
    """Apply learned BPE merges to segment a word"""
    # Start with character-level representation
    symbols = list(word) + ['_']
    
    # Apply each merge in order
    for merge in merges:
        new_symbols = []
        i = 0
        while i < len(symbols):
            if (i < len(symbols) - 1 and 
                symbols[i] == merge[0] and 
                symbols[i + 1] == merge[1]):
                new_symbols.append(merge[0] + merge[1])
                i += 2
            else:
                new_symbols.append(symbols[i])
                i += 1
        symbols = new_symbols
    
    return symbols

print("=== BPE SEGMENTATION EXAMPLES ===\n")

# Test on original words
test_words = ['new', 'newer', 'lowest', 'wider']
for word in test_words:
    segments = apply_bpe_segmentation(word, bpe_learner.merges)
    print(f"'{word}' → {segments}")

# Test on invented word
print(f"\nInvented word:")
invented_segments = apply_bpe_segmentation('newestest', bpe_learner.merges)
print(f"'newestest' → {invented_segments}")

print(f"\nWhy subwords help:")
print("- Represent unseen (OOV) words as compositions of known pieces")
print("- 'newestest' not in training but expressible as subword units")
print("- Morpheme alignment: 'er_' aligns with comparative/agent suffix")


=== BPE SEGMENTATION EXAMPLES ===

'new' → ['new', '_']
'newer' → ['newer_']
'lowest' → ['low', 'e', 's', 't', '_']
'wider' → ['wid', 'er_']

Invented word:
'newestest' → ['new', 'e', 's', 't', 'e', 's', 't', '_']

Why subwords help:
- Represent unseen (OOV) words as compositions of known pieces
- 'newestest' not in training but expressible as subword units
- Morpheme alignment: 'er_' aligns with comparative/agent suffix


## Q3.2: Why use Subword Tokenisation

Subword tokenization effectively solves the out-of-vocabulary (OOV) problem by breaking down unknown words into a sequence of known, smaller pieces. For instance, even though the invented word "newestest" was not in the original corpus, the BPE model could still represent it by segmenting it into subwords like ['new', 'e', 's', 't', 'e', 's', 't', '_']. 

This approach ensures that no word is ever truly "unknown," only a new combination of existing vocabulary parts. A perfect example of subwords aligning with meaningful linguistic units, or morphemes, is the creation of the token er_. This subword directly corresponds to the English comparative suffix, as seen in the segmentation of "newer" and "wider." By learning this common suffix, the model captures a fundamental piece of English grammar, allowing it to better understand and generalize across different words.

Take the word, "faster", as an example for an unknown word, not in the BPE corpus. The token er_ would segment the word by separating the known stem from the suffix, resulting in a tokenization like ['fast', 'er_']. This process allows the model to correctly interpret "faster" as the comparative form of "fast," even if it had never encountered the specific word "faster" during its training, perfectly illustrating its power to generalize.

In [28]:
# Q3.3: Train BPE on the paragraph from Q2 (30 merges)
def prepare_paragraph_for_bpe(text: str):
    """Prepare paragraph for BPE training"""
    # Lowercase and tokenize
    words = text.lower().split()
    
    # Count word frequencies
    word_freqs = Counter(words)
    
    # Convert to character-level with end-of-word markers
    bpe_word_freqs = {}
    for word, freq in word_freqs.items():
        # Remove punctuation and split into characters
        clean_word = ''.join(c for c in word if c.isalnum() or c in "'-")
        if clean_word:
            char_word = ' '.join(list(clean_word)) + ' _'
            bpe_word_freqs[char_word] = freq
    
    return bpe_word_freqs

# Prepare paragraph for BPE
paragraph_bpe_corpus = prepare_paragraph_for_bpe(paragraph)

print("=== BPE ON PARAGRAPH (30 MERGES) ===\n")
print(f"Prepared corpus ({len(paragraph_bpe_corpus)} unique words):")
for word, freq in list(paragraph_bpe_corpus.items())[:10]:  # Show first 10
    print(f"  '{word}': {freq}")
print("  ...")

# Learn BPE on paragraph
paragraph_bpe_learner = BPELearner()
final_paragraph_corpus = paragraph_bpe_learner.learn_bpe(paragraph_bpe_corpus, 30)


=== BPE ON PARAGRAPH (30 MERGES) ===

Prepared corpus (31 unique words):
  'ഇ ന ന ല _': 1
  'എ ന ക ക _': 1
  'റ പ പ ർ ട ട _': 1
  'പ ർ ത ത യ ക ക ൻ _': 1
  'ക ഴ ഞ ഞ ല ല _': 1
  'അ ത _': 2
  'അ ത യ ധ ന ക _': 1
  'മ ഡ ൽ _': 1
  'ആ യ ര ന ന _': 1
  'പ ക ഷ _': 1
  ...
=== BPE LEARNER - 30 MERGES ===

Merge 1: ക + ക -> കക (freq: 6)
Merge 2: ന + ന -> നന (freq: 4)
Merge 3: ല + _ -> ല_ (freq: 4)
Merge 4: ച + ച -> ചച (freq: 4)
Merge 5: ത + യ -> തയ (freq: 3)
Merge 6: ൾ + _ -> ൾ_ (freq: 3)
Merge 7: നന + ല_ -> നനല_ (freq: 2)
Merge 8: കക + _ -> കക_ (freq: 2)
Merge 9: പ + പ -> പപ (freq: 2)
Merge 10: ൻ + _ -> ൻ_ (freq: 2)
Merge 11: ല + ല_ -> ലല_ (freq: 2)
Merge 12: അ + ത -> അത (freq: 2)
Merge 13: അത + _ -> അത_ (freq: 2)
Merge 14: മ + ഡ -> മഡ (freq: 2)
Merge 15: ൽ + _ -> ൽ_ (freq: 2)
Merge 16: യ + ര -> യര (freq: 2)
Merge 17: ക + ഷ -> കഷ (freq: 2)
Merge 18: റ + റ -> ററ (freq: 2)
Merge 19: ന + _ -> ന_ (freq: 2)
Merge 20: പ + ര -> പര (freq: 2)
Merge 21: ണ + _ -> ണ_ (freq: 2)
Merge 22: ഇ + നനല_ -> ഇനനല_ (fr

In [31]:
# Q3.3: Analysis of paragraph BPE results
print("=== PARAGRAPH BPE ANALYSIS ===\n")

# Show top 5 merges
print("Top 5 merges (first 5 applied):")
for i, merge in enumerate(paragraph_bpe_learner.merges[:5], 1):
    print(f"  {i}. {merge[0]} + {merge[1]} -> {''.join(merge)}")

# Find longest subword tokens
all_subwords = set()
for word in final_paragraph_corpus.keys():
    subwords = word.split()
    all_subwords.update(subwords)

# Sort by length and show longest
longest_subwords = sorted(all_subwords, key=len, reverse=True)[:5]
print(f"\nFive longest subword tokens learned:")
print(f"  {longest_subwords}")

# Get top 5 longest words from Malayalam paragraph
malayalam_words = paragraph.split()
# Clean words (remove punctuation) and keep unique words
unique_clean_words = set()
for word in malayalam_words:
    clean_word = ''.join(c for c in word if c.isalnum())
    if clean_word:
        unique_clean_words.add(clean_word.lower())

# Sort by length and get top 5 longest
top_5_longest = sorted(unique_clean_words, key=len, reverse=True)[:5]

# Demonstrate segmentation on top 5 longest Malayalam words
print(f"\nBPE segmentations for top 5 longest Malayalam words:")
for i, word in enumerate(top_5_longest, 1):
    segments = apply_bpe_segmentation(word, paragraph_bpe_learner.merges)
    print(f"  {i}. '{word}' (length: {len(word)}) -> {segments}")

print(f"\nNote: These are the longest words in the Malayalam paragraph, showing how BPE handles complex Malayalam morphology.")

=== PARAGRAPH BPE ANALYSIS ===

Top 5 merges (first 5 applied):
  1. ക + ക -> കക
  2. ന + ന -> നന
  3. ല + _ -> ല_
  4. ച + ച -> ചച
  5. ത + യ -> തയ

Five longest subword tokens learned:
  ['റപപർടട_', 'ഇനനല_', 'എനകക_', 'നനല_', 'അതയ']

BPE segmentations for top 5 longest Malayalam words:
  1. 'പരകഷചചകണടരകകകയണ' (length: 15) -> ['പര', 'കഷ', 'ചച', 'ക', 'ണ', 'ട', 'ര', 'കക', 'ക', 'യ', 'ണ_']
  2. 'പഴയവയകകൾ' (length: 8) -> ['പ', 'ഴ', 'യ', 'വ', 'യ', 'കക', 'ൾ_']
  3. 'സഹകരചചലല' (length: 8) -> ['സ', 'ഹ', 'ക', 'ര', 'ചച', 'ലല_']
  4. 'പർതതയകകൻ' (length: 8) -> ['പർ', 'ത', 'തയ', 'കക', 'ൻ_']
  5. 'എനനരനനല' (length: 7) -> ['എ', 'നന', 'ര', 'നനല_']

Note: These are the longest words in the Malayalam paragraph, showing how BPE handles complex Malayalam morphology.


## Q3.4: Reflection on BPE

**Observation and Reflections from the mini-BPE that we just learned**

The BPE algorithm learned a variety of subword types, reflecting a hierarchy from simple character combinations to more linguistically meaningful units.

- Meaningful Suffixes: The most interesting learned tokens were common Malayalam suffixes, which are true morphemes. For example, ൾ_ (Merge 6) is a standard plural marker, and ൽ_ (Merge 15) often represents the locative case (meaning "in" or "at").

- Whole Words: For very high-frequency words, the algorithm managed to reconstruct the entire word. A great example is അത്_ (ath_, meaning "it" or "that"), which was formed in Merge 13, and ഇന്നലെ_ (innaley_, meaning "yesterday") in Merge 22.

- Common Character Combinations: Many of the initial merges were simply statistically frequent character pairs or consonant clusters, not necessarily meaningful units on their own. Examples include കക (kk), നന (nn), and തയ (thay). These act as basic building blocks for larger tokens.

**NOTE**: *With only 30 merges, the process primarily captured suffixes and very frequent short words. It didn't progress enough to consistently isolate word stems or prefixes.*


**Pros and Cons**

Subword tokenization is a powerful technique for morphologically rich languages like Malayalam, but it comes with trade-offs.

 **Pros** 

   1. Handles Agglutinative Morphology: 
    
      Malayalam words are often formed by adding multiple suffixes to a root stem. Subword tokenization naturally handles this by breaking words into a stem and its constituent suffixes. This allows a model to recognize the relationship between പോകുന്നു (pokunnu - "is going") and പോയി (poyi - "went") by seeing a common stem token, drastically reducing the vocabulary size and helping it generalize to unseen word forms.

   2. Manages Compound Words and Loanwords: 
    
      The method gracefully handles both native compound words (സമാസപദം) and the many English loanwords in Malayalam like "റിപ്പോർട്ട്" (report). Instead of treating "റിപ്പോർട്ട്" as an unknown out-of-vocabulary (OOV) word, BPE breaks it down into manageable subwords (like റപപർടട_ in your output), allowing the model to process it effectively.

 **Cons**

   1. Creates Non-Meaningful Splits: 
    
      Since BPE is driven purely by frequency, it can create subwords that have no linguistic or semantic meaning. For example, splitting a word based on a common but meaningless character pair can make it harder for the model to learn the true compositional meaning of the word. A split might be statistically optimal but linguistically nonsensical.

   2. Struggles with "Sandhi": 
    
      Malayalam uses complex "Sandhi" rules, where sounds at word junctions merge and change (e.g., അതി + ഇൽ → അതിൽ). A BPE model trained on a small corpus might not learn these phonetic rules and could split a merged word in an unnatural way that obscures the original root words, making it difficult for a model to capture the underlying semantics.

# Question 4: Edit Distance

**Task:** Compute edit distance between "Sunday" and "Saturday" using dynamic programming with two different cost models.

This section covers:
1. **Model A:** Substitution = 1, Insertion = 1, Deletion = 1
2. **Model B:** Substitution = 2, Insertion = 1, Deletion = 1
3. **Dynamic programming implementation** with full DP table
4. **Backtracing** to find optimal edit sequences
5. **Analysis and reflection** on cost model effects

**String pair:** `sunday` → `saturday`


In [32]:
# Q4: Edit Distance Implementation
class EditDistance:
    """Edit distance calculator with dynamic programming and backtracing"""
    
    def __init__(self, sub_cost=1, ins_cost=1, del_cost=1):
        self.sub_cost = sub_cost
        self.ins_cost = ins_cost
        self.del_cost = del_cost
    
    def compute_distance(self, src: str, tgt: str):
        """Compute edit distance using dynamic programming"""
        m, n = len(src), len(tgt)
        
        # Initialize DP table
        dp = [[0] * (n + 1) for _ in range(m + 1)]
        
        # Base cases
        for i in range(m + 1):
            dp[i][0] = i * self.del_cost
        for j in range(n + 1):
            dp[0][j] = j * self.ins_cost
        
        # Fill DP table
        for i in range(1, m + 1):
            for j in range(1, n + 1):
                if src[i-1] == tgt[j-1]:
                    # Match - no cost
                    dp[i][j] = dp[i-1][j-1]
                else:
                    # Take minimum of three operations
                    substitute = dp[i-1][j-1] + self.sub_cost
                    insert = dp[i][j-1] + self.ins_cost
                    delete = dp[i-1][j] + self.del_cost
                    dp[i][j] = min(substitute, insert, delete)
        
        return dp, dp[m][n]
    
    def backtrace(self, dp, src: str, tgt: str):
        """Backtrace to find optimal edit sequence"""
        i, j = len(src), len(tgt)
        operations = []
        
        while i > 0 or j > 0:
            if i > 0 and j > 0 and src[i-1] == tgt[j-1]:
                # Match
                operations.append(f"match {src[i-1]} → {tgt[j-1]}")
                i -= 1
                j -= 1
            elif (i > 0 and j > 0 and 
                  dp[i][j] == dp[i-1][j-1] + self.sub_cost):
                # Substitution
                operations.append(f"substitute {src[i-1]} → {tgt[j-1]}")
                i -= 1
                j -= 1
            elif j > 0 and dp[i][j] == dp[i][j-1] + self.ins_cost:
                # Insertion
                operations.append(f"insert {tgt[j-1]}")
                j -= 1
            elif i > 0 and dp[i][j] == dp[i-1][j] + self.del_cost:
                # Deletion
                operations.append(f"delete {src[i-1]}")
                i -= 1
        
        return list(reversed(operations))

# Define the strings
src_string = "sunday"
tgt_string = "saturday"

print(f"Q4 - Edit Distance: '{src_string}' → '{tgt_string}'")
print(f"Source length: {len(src_string)}, Target length: {len(tgt_string)}")


Q4 - Edit Distance: 'sunday' → 'saturday'
Source length: 6, Target length: 8


In [33]:
# Q4.1: Model A (Sub=1, Ins=1, Del=1)
print("=== MODEL A: Sub=1, Ins=1, Del=1 ===\n")

model_a = EditDistance(sub_cost=1, ins_cost=1, del_cost=1)
dp_a, distance_a = model_a.compute_distance(src_string, tgt_string)

print(f"Minimum edit distance: {distance_a}")

# Display DP table
def display_dp_table(dp, src, tgt, title):
    """Display the DP table in a readable format"""
    print(f"\n{title}")
    print("DP Table:")
    
    # Header
    header = "    ε  " + "  ".join(f"{c:>2}" for c in tgt)
    print(header)
    
    # Rows
    for i, row in enumerate(dp):
        if i == 0:
            row_label = "ε"
        else:
            row_label = src[i-1]
        row_str = f"{row_label:>2}: " + "  ".join(f"{val:>2}" for val in row)
        print(row_str)

display_dp_table(dp_a, src_string, tgt_string, "Model A DP Table")

# Backtrace
operations_a = model_a.backtrace(dp_a, src_string, tgt_string)
print(f"\nOptimal edit sequence (one possible path):")
for i, op in enumerate(operations_a, 1):
    print(f"  {i}. {op}")

# Count operation types
op_counts_a = {"match": 0, "substitute": 0, "insert": 0, "delete": 0}
for op in operations_a:
    for op_type in op_counts_a:
        if op.startswith(op_type):
            op_counts_a[op_type] += 1

print(f"\nOperation breakdown:")
for op_type, count in op_counts_a.items():
    if count > 0:
        print(f"  {op_type.capitalize()}: {count}")

print(f"\nCost breakdown: {op_counts_a['insert']}×{model_a.ins_cost} + {op_counts_a['substitute']}×{model_a.sub_cost} = {distance_a}")


=== MODEL A: Sub=1, Ins=1, Del=1 ===

Minimum edit distance: 3

Model A DP Table
DP Table:
    ε   s   a   t   u   r   d   a   y
 ε:  0   1   2   3   4   5   6   7   8
 s:  1   0   1   2   3   4   5   6   7
 u:  2   1   1   2   2   3   4   5   6
 n:  3   2   2   2   3   3   4   5   6
 d:  4   3   3   3   3   4   3   4   5
 a:  5   4   3   4   4   4   4   3   4
 y:  6   5   4   4   5   5   5   4   3

Optimal edit sequence (one possible path):
  1. match s → s
  2. insert a
  3. insert t
  4. match u → u
  5. substitute n → r
  6. match d → d
  7. match a → a
  8. match y → y

Operation breakdown:
  Match: 5
  Substitute: 1
  Insert: 2

Cost breakdown: 2×1 + 1×1 = 3


In [34]:
# Q4.2: Model B (Sub=2, Ins=1, Del=1)
print("=== MODEL B: Sub=2, Ins=1, Del=1 ===\n")

model_b = EditDistance(sub_cost=2, ins_cost=1, del_cost=1)
dp_b, distance_b = model_b.compute_distance(src_string, tgt_string)

print(f"Minimum edit distance: {distance_b}")

display_dp_table(dp_b, src_string, tgt_string, "Model B DP Table")

# Backtrace
operations_b = model_b.backtrace(dp_b, src_string, tgt_string)
print(f"\nOptimal edit sequence (one possible path):")
for i, op in enumerate(operations_b, 1):
    print(f"  {i}. {op}")

# Count operation types
op_counts_b = {"match": 0, "substitute": 0, "insert": 0, "delete": 0}
for op in operations_b:
    for op_type in op_counts_b:
        if op.startswith(op_type):
            op_counts_b[op_type] += 1

print(f"\nOperation breakdown:")
for op_type, count in op_counts_b.items():
    if count > 0:
        print(f"  {op_type.capitalize()}: {count}")

total_cost = (op_counts_b['insert'] * model_b.ins_cost + 
              op_counts_b['substitute'] * model_b.sub_cost + 
              op_counts_b['delete'] * model_b.del_cost)
print(f"\nCost breakdown: {op_counts_b['insert']}×{model_b.ins_cost} + {op_counts_b['substitute']}×{model_b.sub_cost} + {op_counts_b['delete']}×{model_b.del_cost} = {total_cost}")


=== MODEL B: Sub=2, Ins=1, Del=1 ===

Minimum edit distance: 4

Model B DP Table
DP Table:
    ε   s   a   t   u   r   d   a   y
 ε:  0   1   2   3   4   5   6   7   8
 s:  1   0   1   2   3   4   5   6   7
 u:  2   1   2   3   2   3   4   5   6
 n:  3   2   3   4   3   4   5   6   7
 d:  4   3   4   5   4   5   4   5   6
 a:  5   4   3   4   5   6   5   4   5
 y:  6   5   4   5   6   7   6   5   4

Optimal edit sequence (one possible path):
  1. match s → s
  2. insert a
  3. insert t
  4. match u → u
  5. substitute n → r
  6. match d → d
  7. match a → a
  8. match y → y

Operation breakdown:
  Match: 5
  Substitute: 1
  Insert: 2

Cost breakdown: 2×1 + 1×2 + 0×1 = 4


In [35]:
# Q4.3: Comparison of the two models
print("=== COMPARISON OF MODELS A & B ===\n")

print(f"Model A (Sub=1, Ins=1, Del=1): Distance = {distance_a}")
print(f"Model B (Sub=2, Ins=1, Del=1): Distance = {distance_b}")
print(f"Difference: {distance_b - distance_a}")

print(f"\nAlignment comparison:")
print(f"Model A operations: {len(operations_a)} total")
print(f"Model B operations: {len(operations_b)} total")

print(f"\nModel A sequence:")
for op in operations_a:
    print(f"  {op}")

print(f"\nModel B sequence:")
for op in operations_b:
    print(f"  {op}")

# Check if sequences are the same
same_sequence = operations_a == operations_b
print(f"\nSame edit sequence? {same_sequence}")

if same_sequence:
    print("The algorithms found the same optimal alignment.")
    print("The difference in distance is due to different substitution costs.")
else:
    print("The algorithms found different optimal alignments.")
    print("This shows how cost models can affect the preferred edit path.")


=== COMPARISON OF MODELS A & B ===

Model A (Sub=1, Ins=1, Del=1): Distance = 3
Model B (Sub=2, Ins=1, Del=1): Distance = 4
Difference: 1

Alignment comparison:
Model A operations: 8 total
Model B operations: 8 total

Model A sequence:
  match s → s
  insert a
  insert t
  match u → u
  substitute n → r
  match d → d
  match a → a
  match y → y

Model B sequence:
  match s → s
  insert a
  insert t
  match u → u
  substitute n → r
  match d → d
  match a → a
  match y → y

Same edit sequence? True
The algorithms found the same optimal alignment.
The difference in distance is due to different substitution costs.



## Q4.4: Reflection on Edit Distance 

**Reflection on Edit Distance Model Comparison**

* The two models produced different distances 
    - Model B assigns a higher cost of 2 for a substitution, while Model A's cost is only 1.
* For transforming "Sunday" to "Saturday," the most useful operations were two insertions and one substitution.
* The choice of model is critical for different applications:
    * **Spell Check:** For spell-checking tasks where single-character substitutions are common typos, a low substitution cost makes sense. Model A's equal costs are generally better, as common typos like substitutions, insertions, or deletions are treated as equally probable errors.
    * **DNA Alignment:**  In genetics, a substitution (a point mutation) is a fundamentally different biological event than an insertion or deletion (an indel). In DNA sequence alignment, costs should reflect real biological mutation rates. Model B's variable costs are superior, as they can represent the different biological probabilities of a mutation (substitution) versus an indel (insertion/deletion).